# [Module 1.2] 로컬 모드 및 스크립트 모드로 훈련 (SageMaker 사용)

### 본 워크샵의 모든 노트북은 `conda_python3` 여기에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.
- 1. 환경 셋업
- 2. 세이지 메이크 로컬 모드 훈련
- 3. SageMaker Host Mode 로 훈련
- 4. 모델 아티펙트 저장

---

# 1. 환경 셋업

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

In [3]:
### 커스텀 라이브러리
import config 

버킷 및 폴더(prefix) 설정

In [4]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

use_default_bucket = True
if use_default_bucket:
    bucket = sagemaker_session.default_bucket()
else:
    bucket = '<Type your bucket>'
    
prefix = "NCFModel"



# 2. 세이지 메이크 로컬 모드 훈련
#### 로컬의 GPU, CPU 여부로 instance_type 결정

In [5]:
import os
import subprocess


try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        instance_type = "local_gpu"
    else:
        instance_type = "local"        
except:
    pass

print("Instance type = " + instance_type)

Instance type = local_gpu


## 2.1. 스크립트 모드의 코드 작성 방법
- ![script_mode_example](img/script_mode_example.png)

## 2.2.훈련 코드 확인
- 아래의 코드는 전형적인 스크립트 모드의 코드 작성 방법을 따르고 있습니다.
- 훈련 함수는 `from train_lib import train` 로서 이전 노트북의 **[세이지 메이커 없이]** 작성한 스크래치 버전에서 사용한 훈련 함수와 동일 합니다.


In [6]:
train_code = 'src/train.py'
!pygmentize {train_code}

import argparse
import os
import json
import sys
sys.path.append('./src')

from train_lib import train

if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    ##################################
    #### 세이지 메이커 프레임워크의 도커 컨테이너 환경 변수 인자
    ##################################

    parser.add_argument('--train-data-dir', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    
    parser.add_argument('--test-data-dir', type=str, default=os.environ['SM_CHANNEL_TEST'])

    
        
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])    
           
    
    ##################################
    #### 사용자 정의 커맨드 인자
    ##################################

    parser.add_argument("--lr", 
        type=float, 
        default=0.001, 
        help="learning rate")
    parser.add_argument("--dropout", 
        type=float,
        default=0.0,  
        help="dropout rate")
    parser.add_argument("--batch_size", 
        type=int, 
        default=2

## 2.3. 로컬에 있는 데이타 세트의 위치를 지정 합니다.

In [8]:
local_inputs = config.main_path
print("local_inputs: ", local_inputs)

local_inputs:  ../data/


In [9]:
local_inputs = {'train': f'file://{local_inputs}',
          'test': f'file://{local_inputs}'}

## 2.4. 로컬 모드로 훈련 실행
- 아래의 두 라인이 로컬모드로 훈련을 지시 합니다.
```python
    instance_type=instance_type, # local_gpu or local 지정
    session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.
```

In [10]:
hyperparameters = {'epochs': 1, 
                   'lr': 0.001,
                   'batch_size': 256,
                   'top_k' : 10,
                   'dropout' : 0.0,
                   'factor_num' : 32,
                   'num_layers' : 3,
                   'num_ng' : 4,
                   'test_num_ng' : 99,                   
                    }  

In [11]:
from sagemaker.pytorch import PyTorch
import os
import subprocess


local_estimator = PyTorch(
    entry_point="train.py",    
    source_dir='src',    
    role=role,
    framework_version='1.8.1',
    py_version='py3',
    instance_count=1,
    instance_type=instance_type, # local_gpu or local 지정
    session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.
    hyperparameters= hyperparameters               
    
)
local_estimator.fit(local_inputs)

Creating cewpqtd5wr-algo-1-1ticz ... 
Creating cewpqtd5wr-algo-1-1ticz ... done
Attaching to cewpqtd5wr-algo-1-1ticz
cewpqtd5wr-algo-1-1ticz | 2022-05-21 13:37:11,710 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
cewpqtd5wr-algo-1-1ticz | 2022-05-21 13:37:11,733 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
cewpqtd5wr-algo-1-1ticz | 2022-05-21 13:37:11,735 sagemaker_pytorch_container.training INFO     Invoking user training script.
cewpqtd5wr-algo-1-1ticz | 2022-05-21 13:37:11,928 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
cewpqtd5wr-algo-1-1ticz | /opt/conda/bin/python3.6 -m pip install -r requirements.txt
cewpqtd5wr-algo-1-1ticz | Requirement already satisfied: nvidia-ml-py3==7.352 in /opt/conda/lib/python3.6/site-packages (from -r requirements.txt (line 1)) (7.352.0)
cewpqtd5wr-algo-1-1ticz | Collecting pandas==0.24.2
cewpqtd5wr-algo-1-1ticz |   Downloading

# 3. SageMaker Host Mode 로 훈련
- instance_type, session 을 수정 합니다.
- 입력 데이터를 inputs로서 S3 의 경로를 제공합니다.
- wait=False 로 지정해서 async 모드로 훈련을 실행합니다. 
- 실행 경과는 아래의 cifar10_estimator.logs() 에서 확인 합니다.

## 3.1. 데이터 세트를 S3에 업로드


In [13]:
s3_data_loc = sagemaker_session.upload_data(path=config.main_path, bucket=bucket, 
                                       key_prefix=f"{prefix}/data")
print("s3_data_loc: ", s3_data_loc)

s3_data_loc:  s3://sagemaker-us-east-1-057716757052/NCFModel/data


In [14]:
! aws s3 ls {s3_data_loc} --recursive

2022-05-21 07:03:58     128039 NCFModel/data/.ipynb_checkpoints/ml-1m.test-checkpoint.rating
2022-05-21 13:40:46    2891424 NCFModel/data/ml-1m.test.negative
2022-05-21 13:40:43     128039 NCFModel/data/ml-1m.test.rating
2022-05-21 13:40:44   20982911 NCFModel/data/ml-1m.train.rating
2022-05-21 07:03:58   27404899 NCFModel/data/pinterest-20.test.negative
2022-05-21 07:03:54     807267 NCFModel/data/pinterest-20.test.rating
2022-05-21 07:03:55   21138451 NCFModel/data/pinterest-20.train.rating


## 3.2. 훈련 및 테스트 데이터를 S3 로 지정

In [16]:
s3_inputs = {
            'train': f'{s3_data_loc}',
            'test': f'{s3_data_loc}'
            }

print("s3_inputs: \n", s3_inputs)

s3_inputs: 
 {'train': 's3://sagemaker-us-east-1-057716757052/NCFModel/data', 'test': 's3://sagemaker-us-east-1-057716757052/NCFModel/data'}


## 3.3 훈련 실행
- epochs 값을 조절해서 실행 시간을 조정 하세요.

In [17]:
host_hyperparameters = {'epochs': 10, 
                       'lr': 0.001,
                       'batch_size': 256,
                       'top_k' : 10,
                       'dropout' : 0.0,
                       'factor_num' : 32,
                       'num_layers' : 3,
                       'num_ng' : 4,
                       'test_num_ng' : 99,                   
                    }  

In [18]:
from sagemaker.pytorch import PyTorch

instance_type = 'ml.p3.2xlarge'

host_estimator = PyTorch(
    entry_point="train.py",    
    source_dir='src',    
    role=role,
    framework_version='1.8.1',
    py_version='py3',
    instance_count=1,
    instance_type=instance_type,
    session = sagemaker.Session(), # 세이지 메이커 세션
    hyperparameters=host_hyperparameters
    
)
host_estimator.fit(s3_inputs, wait=False)

In [27]:
%%time

host_estimator.logs()

2022-05-21 14:50:07 Starting - Preparing the instances for training
2022-05-21 14:50:07 Downloading - Downloading input data
2022-05-21 14:50:07 Training - Training image download completed. Training in progress.
2022-05-21 14:50:07 Uploading - Uploading generated training model
2022-05-21 14:50:07 Completed - Training job completedbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-05-21 13:48:40,935 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-05-21 13:48:40,958 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-05-21 13:48:40,966 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-05-21 13:48:41,447 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt
  Attempting uninstall: numpy
    Found existing installat

# 4. 모델 아티펙트 저장
- S3 에 저장된 모델 아티펙트를 저장하여 추론시 사용합니다.

In [28]:
artifact_path = host_estimator.model_data
print("artifact_path: ", artifact_path)

%store artifact_path

artifact_path:  s3://sagemaker-us-east-1-057716757052/pytorch-training-2022-05-21-13-41-33-332/output/model.tar.gz
Stored 'artifact_path' (str)


기타 변수 저장

In [29]:
%store bucket 
%store prefix

Stored 'bucket' (str)
Stored 'prefix' (str)
